In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="lJQDF0kQizvYcyEk5DcH")
project = rf.workspace("selfdriving-car-qtywx").project("self-driving-cars-lfjou")
version = project.version(6)
dataset = version.download("yolokeras")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Self-Driving-Cars-6 in yolokeras:: 100%|██████████| 4980/4980 [00:01<00:00, 2849.44it/s]


In [4]:
# prompt: detect car violating red signal

# Import necessary libraries
import os
from roboflow import Roboflow
from inference_sdk import InferenceHTTPClient
from google.colab import drive
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random

# Install necessary libraries (if not already installed)
!pip install roboflow
!pip install inference-sdk

# --- Initialize Roboflow and InferenceHTTPClient ---
rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY")  # Replace with your actual API key
project = rf.workspace("YOUR_ROBOFLOW_WORKSPACE").project("YOUR_ROBOFLOW_PROJECT")
version = project.version(YOUR_ROBOFLOW_VERSION) # Replace with your project version

# Initialize the client for inference
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="YOUR_ROBOFLOW_API_KEY" # Replace with your actual API key
)


# --- Mount Google Drive ---
drive.mount('/content/drive')


# --- Function to detect red signal violations ---
def detect_red_light_violation(image_path):
    """
    Detects if a car is violating a red light in an image.

    Args:
        image_path: Path to the image file.

    Returns:
        A dictionary containing the inference results and a boolean indicating a violation.
    """

    try:
        result = CLIENT.infer(image_path, model_id=f"{project.name}/{version.version}")

        # Process the inference result to check for red light violations
        # You need to inspect the 'predictions' field of the result object
        # and define your criteria for a red light violation

        violation_detected = False
        for prediction in result['predictions']:
            if prediction['class'] == "red_light" and prediction['confidence'] > 0.8: # Example
                # Customize this condition to fit your needs
                violation_detected = True
                break

        return {"result": result, "violation": violation_detected}
    except Exception as e:
        print(f"An error occurred during inference: {e}")
        return {"result": None, "violation": False} # Return false on error

# --- Example usage ---
image_path = "/content/drive/My Drive/Your/Image/Path/image.jpg" # Replace with the path to your image
result = detect_red_light_violation(image_path)

if result["violation"]:
    print(f"Red light violation detected in image: {image_path}")
    # Further actions (e.g. save the image, send an alert, etc.)
else:
    print(f"No red light violation detected in image: {image_path}")

#Display the image for visualization
img = Image.open(image_path)
plt.imshow(img)
plt.show()

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=undefined&ref=undefined
loading Roboflow workspace...


RoboflowError: {
    "error": {
        "message": "Unsupported get request. Workspace with ID \"YOUR_ROBOFLOW_WORKSPACE\" does not exist or cannot be loaded due to missing permissions.",
        "status": 404,
        "type": "GraphMethodException",
        "hint": "You can see your available workspaces by issuing a GET request to /workspaces"
    }
}

In [ ]:
# prompt: use deep learning embeddings and run k mean for grouping the test set

import numpy as np
from sklearn.cluster import KMeans
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import os

# ... (Your existing code for Roboflow, data loading, etc.)

# --- Extract Embeddings ---
def extract_embeddings(image_paths):
    """Extracts embeddings from images using ResNet50."""
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')  # Use average pooling
    embeddings = []
    for image_path in image_paths:
        try:
          img = image.load_img(image_path, target_size=(224, 224))
          x = image.img_to_array(img)
          x = np.expand_dims(x, axis=0)
          x = preprocess_input(x)
          embedding = model.predict(x).flatten() # Flatten to get a 1D embedding
          embeddings.append(embedding)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            embeddings.append(np.zeros(2048)) # Append zero vector in case of error
    return np.array(embeddings)

# --- Example usage ---

# Get all images from the test directory
test_image_paths = [os.path.join(valid_dir, filename) for filename in os.listdir(valid_dir) if filename.endswith(('.jpg', '.jpeg', '.png'))]

# Extract embeddings for test images
test_embeddings = extract_embeddings(test_image_paths)

# --- Run K-means clustering ---
n_clusters = 3  # Number of clusters (adjust as needed)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(test_embeddings)

# --- Print cluster assignments ---
for i, image_path in enumerate(test_image_paths):
  print(f"Image: {image_path}, Cluster: {cluster_labels[i]}")


# --- (Optional) Visualize the clusters ---
# You could use matplotlib or another library to display a sample of images from each cluster
# to visualize the results.

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1

In [ ]:
# --- Run K-means clustering ---
n_clusters = 3  # Number of clusters (adjust as needed)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(test_embeddings)

# --- Print cluster assignments ---
for i, image_path in enumerate(test_image_paths):
  print(f"Image: {image_path}, Cluster: {cluster_labels[i]}")

Image: /content/drive/My Drive/Colab Notebooks/dataset-robo/valid/FisheyeCamera_1_00009_png.rf.444f540be8f12473bb40e4117a96a833.jpg, Cluster: 1
Image: /content/drive/My Drive/Colab Notebooks/dataset-robo/valid/000047_jpg.rf.3da4d486d7b29ef66899b32813c47247.jpg, Cluster: 1
Image: /content/drive/My Drive/Colab Notebooks/dataset-robo/valid/00007_00002_00011_png.rf.3e6ecd510efe187683f4efffca1642f0.jpg, Cluster: 0
Image: /content/drive/My Drive/Colab Notebooks/dataset-robo/valid/FisheyeCamera_1_00090_png.rf.45934e8b7db1097868870f3cc5c1960e.jpg, Cluster: 1
Image: /content/drive/My Drive/Colab Notebooks/dataset-robo/valid/00014_00001_00000_png.rf.3e50b1a77c9ffef0b94e6a561cc319c8.jpg, Cluster: 2
Image: /content/drive/My Drive/Colab Notebooks/dataset-robo/valid/FisheyeCamera_1_00605_png.rf.423e1629b3a0645c38f8555de3ad1d2b.jpg, Cluster: 1
Image: /content/drive/My Drive/Colab Notebooks/dataset-robo/valid/00001_00005_00008_png.rf.40d0da91ea6cc55d098dfc3e32176ed4.jpg, Cluster: 2
Image: /content/dri

In [ ]:
# prompt: load traffic_violaions from google drive and perform nlp analysis

# Install necessary libraries
!pip install roboflow
!pip install inference-sdk
!pip install tensorflow
!pip install scikit-learn
!pip install Pillow

# Import libraries
from roboflow import Roboflow
from inference_sdk import InferenceHTTPClient
import os
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from PIL import Image
import random
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

# Mount Google Drive
drive.mount('/content/drive')

# --- Data preprocessing (example) ---
IMG_WIDTH = 224
IMG_HEIGHT = 224

def preprocess_image(image_path):
    img = Image.open(image_path).resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = np.array(img) / 255.0
    return img_array

# Load and preprocess images
def load_data(directory):
    images = []
    labels = []  # Placeholder for labels
    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(directory, filename)
            try:
                images.append(preprocess_image(image_path))
                # Replace with actual label extraction logic
                labels.append(0)
            except Exception as e:
                print(f"Error processing image {filename}: {e}")
    return np.array(images), np.array(labels)

# Define paths
dataset_path = "/content/drive/My Drive/Colab Notebooks/dataset-robo" # Update with your actual path
train_dir = os.path.join(dataset_path, "train")
valid_dir = os.path.join(dataset_path, "valid")

X_train, y_train = load_data(train_dir)
X_val, y_val = load_data(valid_dir)

# Split data
split_ratio = 0.8
split_index = int(len(X_train) * split_ratio)
X_train_split = X_train[:split_index]
y_train_split = y_train[:split_index]
X_test = X_train[split_index:]
y_test = y_train[split_index:]


# Define and train a CNN model (example)
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid') # Adjust output layer based on task
])

model.compile(optimizer='adam',
              loss='binary_crossentropy', # Adjust loss function accordingly
              metrics=['accuracy'])

model.fit(X_train_split, y_train_split, epochs=10, validation_data=(X_val, y_val))


# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


# --- Extract Embeddings using ResNet50 ---
def extract_embeddings(image_paths):
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    embeddings = []
    for image_path in image_paths:
        try:
            img = image.load_img(image_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            embedding = model.predict(x).flatten()
            embeddings.append(embedding)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            embeddings.append(np.zeros(2048)) # Handle errors gracefully
    return np.array(embeddings)


# Example usage (get image paths, extract embeddings)
test_image_paths = [os.path.join(valid_dir, filename) for filename in os.listdir(valid_dir) if filename.endswith(('.jpg', '.jpeg', '.png'))]
test_embeddings = extract_embeddings(test_image_paths)


# --- K-means Clustering ---
n_clusters = 3 # Adjust number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(test_embeddings)

# Print cluster assignments
for i, image_path in enumerate(test_image_paths):
    print(f"Image: {image_path}, Cluster: {cluster_labels[i]}")

In [ ]:
# prompt: load traffic_violaions.csv from drive and Colab Notebooks

import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the CSV file from Google Drive
file_path = '/content/drive/My Drive/Colab Notebooks/traffic_violaions.csv'  # Replace with the actual path

try:
    df = pd.read_csv(file_path)
    print(df.head())  # Display the first few rows of the DataFrame
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except pd.errors.ParserError:
    print(f"Error: Could not parse the CSV file at {file_path}. Check the file format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Mounted at /content/drive
   stop_date stop_time  country_name driver_gender  driver_age_raw  \
0   1/2/2005      1:55           NaN             M          1985.0   
1  1/18/2005      8:15           NaN             M          1965.0   
2  1/23/2005     23:15           NaN             M          1972.0   
3  2/20/2005     17:15           NaN             M          1986.0   
4  3/14/2005     10:00           NaN             F          1984.0   

   driver_age driver_race     violation_raw violation search_conducted  \
0        20.0       White          Speeding  Speeding            False   
1        40.0       White          Speeding  Speeding            False   
2        33.0       White          Speeding  Speeding            False   
3        19.0       White  Call for Service     Other            False   
4        21.0       White          Speeding  Speeding            False   

  search_type   stop_outcome is_arrested stop_duration drugs_related_stop  
0         NaN       Citation    

In [ ]:
import pandas as pd

# ... (Rest of your code)

def identify_violation(violation_type):  # Using 'violation_type' column
    # Adjust the logic based on the values in 'violation_type'
    keywords = ["speeding", "red light", "stop sign", "parking"]
    for keyword in keywords:
        if keyword in violation_type.lower():
            return True
    return False

df["violation"] = df["violation_raw"].apply(identify_violation)  # Updated line

# ... (Rest of your code)

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
import pandas as pd

# ... (Rest of your code)

def identify_violation(violation_type):  # Using 'violation_type' column
    # Adjust the logic based on the values in 'violation_type'

    # Handle missing or non-string values
    if isinstance(violation_type, str):
        keywords = ["speeding", "red light", "stop sign", "parking"]
        for keyword in keywords:
            if keyword in violation_type.lower():
                return True
    return False  # Return False for non-string or missing values


df["violation"] = df["violation_raw"].apply(identify_violation)  # Updated line

# ... (Rest of your code)

In [ ]:
# prompt: perform nlp analysis

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import pandas as pd

# ... (Your existing code)

# Load the CSV file from Google Drive (assuming it's already loaded as 'df')

def analyze_violation_text(text):
    if isinstance(text, str): # Check if text is a string
        tokens = nltk.word_tokenize(text)
        tagged = nltk.pos_tag(tokens)
        entities = nltk.ne_chunk(tagged)

        # Basic NLP analysis (customize further for your needs)
        violation_keywords = ["speeding", "red light", "stop sign", "parking"]
        location_keywords = ["intersection", "street", "highway", "avenue"]

        violation_detected = any(keyword in text.lower() for keyword in violation_keywords)
        location_mentioned = any(keyword in text.lower() for keyword in location_keywords)

        return {"violation_detected": violation_detected, "location_mentioned": location_mentioned, "entities": entities}
    else:
        return {"violation_detected": False, "location_mentioned": False, "entities": None}

df["nlp_analysis"] = df["violation_raw"].apply(analyze_violation_text)


# Print or analyze the results
for index, row in df.iterrows():
    analysis = row["nlp_analysis"]
    print(f"Violation text: {row['violation_raw']}")
    print(f"Violation detected: {analysis['violation_detected']}")
    print(f"Location mentioned: {analysis['location_mentioned']}")

    if analysis["entities"]:
        print("Entities found:")
        for entity in analysis["entities"]:
            if hasattr(entity, 'label'):
                print(f"Label: {entity.label()}, Text: {' '.join(c[0] for c in entity.leaves())}")
    print("-" * 20)


# Further analysis and data visualization using pandas and matplotlib (if needed)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# prompt: perform nlp analysis

import nltk

# Download the necessary NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
# Download the 'punkt_tab' resource
nltk.download('punkt_tab') # Add this line to download the necessary resource


import pandas as pd

# ... (Your existing code)

# Load the CSV file from Google Drive (assuming it's already loaded as 'df')

def analyze_violation_text(text):
    if isinstance(text, str): # Check if text is a string
        tokens = nltk.word_tokenize(text)
        tagged = nltk.pos_tag(tokens)
        entities = nltk.ne_chunk(tagged)

        # Basic NLP analysis (customize further for your needs)
        violation_keywords = ["speeding", "red light", "stop sign", "parking"]
        location_keywords = ["intersection", "street", "highway", "avenue"]

        violation_detected = any(keyword in text.lower() for keyword in violation_keywords)
        location_mentioned = any(keyword in text.lower() for keyword in location_keywords)

        return {"violation_detected": violation_detected, "location_mentioned": location_mentioned, "entities": entities}
    else:
        return {"violation_detected": False, "location_mentioned": False, "entities": None}

df["nlp_analysis"] = df["violation_raw"].apply(analyze_violation_text)


# Print or analyze the results
for index, row in df.iterrows():
    analysis = row["nlp_analysis"]
    print(f"Violation text: {row['violation_raw']}")
    print(f"Violation detected: {analysis['violation_detected']}")
    print(f"Location mentioned: {analysis['location_mentioned']}")

    if analysis["entities"]:
        print("Entities found:")
        for entity in analysis["entities"]:
            if hasattr(entity, 'label'):
                print(f"Label: {entity.label()}, Text: {' '.join(c[0] for c in entity.leaves())}")
    print("-" * 20)


# Further analysis and data visualization using pandas and matplotlib (if needed)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# prompt: perform nlp analysis

import nltk

# Download the necessary NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt_tab')
# Add this to download the language model
nltk.download('averaged_perceptron_tagger_eng')


import pandas as pd

# ... (Your existing code)

# Load the CSV file from Google Drive (assuming it's already loaded as 'df')

def analyze_violation_text(text):
    if isinstance(text, str): # Check if text is a string
        tokens = nltk.word_tokenize(text)
        tagged = nltk.pos_tag(tokens)
        entities = nltk.ne_chunk(tagged)

        # Basic NLP analysis (customize further for your needs)
        violation_keywords = ["speeding", "red light", "stop sign", "parking"]
        location_keywords = ["intersection", "street", "highway", "avenue"]

        violation_detected = any(keyword in text.lower() for keyword in violation_keywords)
        location_mentioned = any(keyword in text.lower() for keyword in location_keywords)

        return {"violation_detected": violation_detected, "location_mentioned": location_mentioned, "entities": entities}
    else:
        return {"violation_detected": False, "location_mentioned": False, "entities": None}

df["nlp_analysis"] = df["violation_raw"].apply(analyze_violation_text)


# Print or analyze the results
for index, row in df.iterrows():
    analysis = row["nlp_analysis"]
    print(f"Violation text: {row['violation_raw']}")
    print(f"Violation detected: {analysis['violation_detected']}")
    print(f"Location mentioned: {analysis['location_mentioned']}")

    if analysis["entities"]:
        print("Entities found:")
        for entity in analysis["entities"]:
            if hasattr(entity, 'label'):
                print(f"Label: {entity.label()}, Text: {' '.join(c[0] for c in entity.leaves())}")
    print("-" * 20)


# Further analysis and data visualization using pandas and matplotlib (if needed)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


NameError: name 'df' is not defined

In [ ]:
# prompt: perform nlp analysis

import nltk

# Download the necessary NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt_tab')
# Add this to download the language model
nltk.download('averaged_perceptron_tagger_eng')


import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the traffic_violations.csv file
# Replace 'path/to/traffic_violations.csv' with the actual path to your CSV file
file_path = '/content/drive/MyDrive/Colab Notebooks/traffic_violations.csv' # Updated file path
df = pd.read_csv(file_path) # Create the DataFrame 'df' by reading the CSV file


# ... (Your existing code)

# Load the CSV file from Google Drive (assuming it's already loaded as 'df')

def analyze_violation_text(text):
    if isinstance(text, str): # Check if text is a string
        tokens = nltk.word_tokenize(text)
        tagged = nltk.pos_tag(tokens)
        entities = nltk.ne_chunk(tagged)

        # Basic NLP analysis (customize further for your needs)
        violation_keywords = ["speeding", "red light", "stop sign", "parking"]
        location_keywords = ["intersection", "street", "highway", "avenue"]

        violation_detected = any(keyword in text.lower() for keyword in violation_keywords)
        location_mentioned = any(keyword in text.lower() for keyword in location_keywords)

        return {"violation_detected": violation_detected, "location_mentioned": location_mentioned, "entities": entities}
    else:
        return {"violation_detected": False, "location_mentioned": False, "entities": None}

df["nlp_analysis"] = df["violation_raw"].apply(analyze_violation_text)


# Print or analyze the results
for index, row in df.iterrows():
    analysis = row["nlp_analysis"]
    print(f"Violation text: {row['violation_raw']}")
    print(f"Violation detected: {analysis['violation_detected']}")
    print(f"Location mentioned: {analysis['location_mentioned']}")

    if analysis["entities"]:
        print("Entities found:")
        for entity in analysis["entities"]:
            if hasattr(entity, 'label'):
                print(f"Label: {entity.label()}, Text: {' '.join(c[0] for c in entity.leaves())}")
    print("-" * 20)


# Further analysis and data visualization using pandas and matplotlib (if needed)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


LookupError: 
**********************************************************************
  Resource [93mmaxent_ne_chunker_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('maxent_ne_chunker_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mchunkers/maxent_ne_chunker_tab/english_ace_multiclass/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# prompt: perform nlp analysis

import nltk

# Download the necessary NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt_tab')
# Add this to download the language model
nltk.download('averaged_perceptron_tagger_eng')
# Download the missing 'maxent_ne_chunker_tab' resource
nltk.download('maxent_ne_chunker_tab') # This line downloads the missing data


import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the traffic_violations.csv file
# Replace 'path/to/traffic_violations.csv' with the actual path to your CSV file
file_path = '/content/drive/MyDrive/Colab Notebooks/traffic_violations.csv' # Updated file path
df = pd.read_csv(file_path) # Create the DataFrame 'df' by reading the CSV file


# ... (Your existing code)

# Load the CSV file from Google Drive (assuming it's already loaded as 'df')

def analyze_violation_text(text):
    if isinstance(text, str): # Check if text is a string
        tokens = nltk.word_tokenize(text)
        tagged = nltk.pos_tag(tokens)
        entities = nltk.ne_chunk(tagged)

        # Basic NLP analysis (customize further for your needs)
        violation_keywords = ["speeding", "red light", "stop sign", "parking"]
        location_keywords = ["intersection", "street", "highway", "avenue"]

        violation_detected = any(keyword in text.lower() for keyword in violation_keywords)
        location_mentioned = any(keyword in text.lower() for keyword in location_keywords)

        return {"violation_detected": violation_detected, "location_mentioned": location_mentioned, "entities": entities}
    else:
        return {"violation_detected": False, "location_mentioned": False, "entities": None}

df["nlp_analysis"] = df["violation_raw"].apply(analyze_violation_text)


# Print or analyze the results
for index, row in df.iterrows():
    analysis = row["nlp_analysis"]
    print(f"Violation text: {row['violation_raw']}")
    print(f"Violation detected: {analysis['violation_detected']}")
    print(f"Location mentioned: {analysis['location_mentioned']}")

    if analysis["entities"]:
        print("Entities found:")
        for entity in analysis["entities"]:
            if hasattr(entity, 'label'):
                print(f"Label: {entity.label()}, Text: {' '.join(c[0] for c in entity.leaves())}")
    print("-" * 20)


# Further analysis and data visualization using pandas and matplotlib (if needed)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 